In [1]:
import os

In [2]:
%pwd

'e:\\Nlp Project\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Nlp Project\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-10-25 13:17:17,792: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-25 13:17:17,794: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-25 13:17:17,796: INFO: common: created directory at: artifacts]
[2023-10-25 13:17:17,797: INFO: common: created directory at: artifacts/model_trainer]


e:\Nlp Project\Text-Summarizer-Project\testS\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mayank Sahu\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailym

  0%|          | 0/920 [00:00<?, ?it/s]

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 3.1382, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 3.0794, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}
{'loss': 2.8823, 'learning_rate': 3e-06, 'epoch': 0.03}
{'loss': 2.8975, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.04}
{'loss': 2.8288, 'learning_rate': 5e-06, 'epoch': 0.05}
{'loss': 2.7802, 'learning_rate': 6e-06, 'epoch': 0.07}
{'loss': 2.5162, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.08}
{'loss': 2.6762, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.09}
{'loss': 2.4067, 'learning_rate': 9e-06, 'epoch': 0.1}
{'loss': 2.4668, 'learning_rate': 1e-05, 'epoch': 0.11}
{'loss': 2.2988, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.12}
{'loss': 2.0897, 'learning_rate': 1.2e-05, 'epoch': 0.13}
{'loss': 2.2369, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.14}
{'loss': 2.072, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.15}
{'loss': 1.9949, 'learning_rate': 1.5e-05, 'epoch': 0.16}
{'loss': 2.0631, 'learnin

  0%|          | 0/818 [00:00<?, ?it/s]

{'eval_loss': 1.4841681718826294, 'eval_runtime': 517.3194, 'eval_samples_per_second': 1.581, 'eval_steps_per_second': 1.581, 'epoch': 0.54}
{'loss': 1.6166, 'learning_rate': 4.880952380952381e-05, 'epoch': 0.55}
{'loss': 1.6929, 'learning_rate': 4.761904761904762e-05, 'epoch': 0.56}
{'loss': 1.676, 'learning_rate': 4.642857142857143e-05, 'epoch': 0.58}
{'loss': 1.6149, 'learning_rate': 4.523809523809524e-05, 'epoch': 0.59}
{'loss': 1.6358, 'learning_rate': 4.404761904761905e-05, 'epoch': 0.6}
{'loss': 1.6113, 'learning_rate': 4.2857142857142856e-05, 'epoch': 0.61}
{'loss': 1.576, 'learning_rate': 4.166666666666667e-05, 'epoch': 0.62}
{'loss': 1.6999, 'learning_rate': 4.047619047619048e-05, 'epoch': 0.63}
{'loss': 1.5822, 'learning_rate': 3.928571428571429e-05, 'epoch': 0.64}
{'loss': 1.6939, 'learning_rate': 3.809523809523809e-05, 'epoch': 0.65}
{'loss': 1.6309, 'learning_rate': 3.690476190476191e-05, 'epoch': 0.66}
{'loss': 1.6223, 'learning_rate': 3.571428571428572e-05, 'epoch': 0.6